In [ ]:
# -*- coding: utf-8 -*-
"""
GLFT 做市商模型（T→∞，全以 tick 为单位）
---------------------------------------
单位约定（全部是 tick 维度）：
  - sigma_t : 价格波动率（tick/√时间）
  - k_t     : 成交强度对距离衰减（1/tick），强度 ~ A * exp(-k_t * δ_tick)
  - gamma_t : 风险厌恶（1/tick-of-PnL），CARA：U = -exp(-gamma_t * PnL_in_ticks)
  - δ_b∞、δ_a∞、ψ∞ ：最优距离 / 点差（tick）

重要：如果你手上是“价格单位”的参数（sigma, k, gamma），
      有 tick_size=τ，则：
        sigma_t  = sigma / τ
        k_t      = k * τ
        gamma_t  = gamma * τ
      这样可保持 GLFT 公式完全同形（α=0.5*k_t*gamma_t*sigma_t^2，η同式）。
"""

from dataclasses import dataclass
import numpy as np
from numpy.linalg import eigh


@dataclass
class GLFTTickParams:
    # —— 以 tick 为单位的参数 —— #
    sigma_t: float     # 波动率 [tick/√time]
    A: float           # 成交强度基准 [1/time]
    k_t: float         # 衰减系数 [1/tick]
    gamma_t: float     # 风险厌恶 [1/tick-of-PnL]
    Q: float           # 最大绝对库存（数量单位，不是 tick）
    q_step: float = 1.0  # 最小库存量子 Δq（数量单位，可为小数）
    beta_t: float = 0.0  # 对角线性项系数（扩展用，默认 0）


class GLFTInfiniteHorizonTicks:
    """
    GLFT 无限时域（tick 单位）
    - 构造三对角矩阵 M（尺寸 N = 2Q/Δq + 1）：
        对角：α * q^2 - beta_t * q
        上/下对角：-η
      其中：
        α  = 0.5 * k_t * gamma_t * sigma_t^2
        η  = A * (1 + gamma_t/k_t)^(-(1 + k_t/gamma_t))
    - 取 M 的最小特征值对应向量 f0（取绝对值），在 ln 域插值计算任意实数 q 的 δ。
    - 报价（全为 tick）：
        C_t        = (1/gamma_t) * ln(1 + gamma_t/k_t)
        δ_b∞(q)    = C_t + (1/k_t) * [ ln f(q) - ln f(q+Δq) ]
        δ_a∞(q)    = C_t + (1/k_t) * [ ln f(q) - ln f(q-Δq) ]
        ψ∞(q)      = δ_b∞(q) + δ_a∞(q)
    """

    def __init__(self, p: GLFTTickParams):
        self.p = p
        if self.p.q_step <= 0: raise ValueError("q_step 必须 > 0")
        if self.p.k_t <= 0 or self.p.A <= 0 or self.p.gamma_t <= 0 or self.p.sigma_t < 0:
            raise ValueError("需满足：k_t>0, A>0, gamma_t>0, sigma_t≥0")
        if self.p.Q <= 0: raise ValueError("Q 必须 > 0")

        # 对齐网格
        Q = float(self.p.Q)
        step = float(self.p.q_step)
        n = int(round(2 * Q / step)) + 1
        self.q_min = -step * (n - 1) / 2.0
        self.q_max =  step * (n - 1) / 2.0
        self.step  = step
        self.q_vals = self.q_min + np.arange(n, dtype=float) * step

        # 系数（tick 体系）
        self.alpha = 0.5 * self.p.k_t * self.p.gamma_t * (self.p.sigma_t ** 2)
        self.eta   = self.p.A * (1.0 + self.p.gamma_t / self.p.k_t) ** (-(1.0 + self.p.k_t / self.p.gamma_t))
        self.Ct    = (1.0 / self.p.gamma_t) * np.log(1.0 + self.p.gamma_t / self.p.k_t)

        # 三对角矩阵 M
        diag = self.alpha * (self.q_vals ** 2) - self.p.beta_t * self.q_vals
        off  = -np.full(n - 1, self.eta, dtype=float)
        M = np.diag(diag)
        M[:-1, 1:] += np.diag(off)
        M[1:, :-1] += np.diag(off)

        # 主特征向量
        w, V = eigh(M)
        f0 = np.abs(V[:, np.argmin(w)]) + 1e-300
        self.lnf0 = np.log(f0)

    # —— 内部：分数索引 + ln 域线性插值 —— #
    def _frac_index(self, q: float) -> float:
        qc = min(max(q, self.q_min), self.q_max)
        return (qc - self.q_min) / self.step

    def _lnf_interp(self, i: float) -> float:
        n = self.lnf0.size
        if i <= 0: return float(self.lnf0[0])
        if i >= n - 1: return float(self.lnf0[-1])
        j = int(np.floor(i)); w = i - j
        return float((1 - w) * self.lnf0[j] + w * self.lnf0[j + 1])

    # —— 最优报价（tick） —— #
    def delta_b_infty_ticks(self, q: float) -> float:
        if q >= self.q_max: return float("nan")
        i  = self._frac_index(q)
        i1 = self._frac_index(q + self.step)
        return self.Ct + (1.0 / self.p.k_t) * (self._lnf_interp(i) - self._lnf_interp(i1))

    def delta_a_infty_ticks(self, q: float) -> float:
        if q <= self.q_min: return float("nan")
        i  = self._frac_index(q)
        i1 = self._frac_index(q - self.step)
        return self.Ct + (1.0 / self.p.k_t) * (self._lnf_interp(i) - self._lnf_interp(i1))

    def spread_infty_ticks(self, q: float) -> float:
        db = self.delta_b_infty_ticks(q)
        da = self.delta_a_infty_ticks(q)
        return db + da if np.isfinite(db) and np.isfinite(da) else float("nan")

    # —— 如需价格（以 price 为单位），再乘以 tick_size —— #
    @staticmethod
    def to_price_from_ticks(delta_in_ticks: float, *, tick_size: float) -> float:
        return float(delta_in_ticks) * float(tick_size)

    @staticmethod
    def round_to_tick(price: float, *, tick_size: float) -> float:
        tau = float(tick_size)
        return round(price / tau) * tau

    def quotes_in_price(self, q: float, mid_price: float, *, tick_size: float):
        """
        以“价格”输出最优挂价（按 tick 对齐）：
            bid = round_to_tick(mid - δ_b_tick * τ)
            ask = round_to_tick(mid + δ_a_tick * τ)
        """
        db_tick = self.delta_b_infty_ticks(q)
        da_tick = self.delta_a_infty_ticks(q)
        print("delta_b_ticks:", db_tick)
        print("delta_a_ticks:", da_tick)
        if not np.isfinite(db_tick): bid = float("nan")
        else:
            bid = self.round_to_tick(mid_price - db_tick * tick_size, tick_size=tick_size)
        if not np.isfinite(da_tick): ask = float("nan")
        else:
            ask = self.round_to_tick(mid_price + da_tick * tick_size, tick_size=tick_size)
        return dict(bid=bid, ask=ask, db_ticks=db_tick, da_ticks=da_tick)


# ===== 用法示例（tick 单位） =====
if __name__ == "__main__":
    # 假设你有“价格单位”的参数，也可以一键转换到 tick 体系
    tick_size = 0.01
    p = GLFTTickParams(
        sigma_t=4,   # price/√time
        A=0.19,
        k_t=0.66,          # 1/price
        gamma_t=0.001,     # 1/price-of-PnL
        Q=20.0,
        q_step=0.1,
    )

    mm = GLFTInfiniteHorizonTicks(p)


    # 若需要“价格”报价（并按 tick 对齐到牌价）
    mid = 180.00
    print(mm.quotes_in_price(q=1, mid_price=mid, tick_size=tick_size))


delta_b_ticks: 1.653491096994917
delta_a_ticks: 1.3897605835434923
{'bid': 179.98, 'ask': 180.01, 'db_ticks': np.float64(1.653491096994917), 'da_ticks': np.float64(1.3897605835434923)}
